In [1]:
import numpy as np
class SVM:
    def __init__(self,C=1,kernel='rbf',gamma=1,tol=1e-3,max_iter=500):
        self.C=C
        self.kernel=kernel
        self.tol=tol
        self.max_iter=max_iter
        if self.kernel in ['rbf','laplace']:
            self.kTup=(self.kernel,gamma)
        elif self.kernel=='linear':
            self.kTup=(self.kernel,1)
        
    def fit(self,X,y):
        self.m,self.n=X.shape
        self.X=X
        self.y=2*y-1   
        self.alphas = np.zeros((self.m,1))
        self.b = 0
        self.eCache = np.zeros((self.m,2)) 
        self.K = np.zeros((self.m,self.m))
        for i in range(self.m):
            self.K[:,i] = type(self).kernelTrans(self.X, self.X[i], self.kTup)
        
        self.intercept_,self.alphas=self.smoP()
        self.support_=np.where(self.alphas>0)[0]
        self.n_support_=len(self.support_)
        self.effective_alpha_=self.alphas[self.alphas>0]
        self.support_vectors_=X[self.support_]
        self.support_labels_=self.y[self.support_]
        self.coef_=self.calcWs()
        self.intercept_=(self.support_labels_[0]-(self.alphas.T* self.y @self.K[:,self.support_[0]]))[0]
    
        return self
    
    def predict(self,X):
        m,n = X.shape
        pred=[]
        for i in range(m):
            K = type(self).kernelTrans(self.support_vectors_,X[i],self.kTup)
            label=((K*self.support_labels_)@self.effective_alpha_+self.intercept_)  
            pred.append(label)
        pred=[0 if i<=0 else 1 for i in np.array(pred).flatten()]
        return np.array(pred)
        
    def clipAlpha(aj,H,L):
        if aj > H: 
            aj = H
        if L > aj:
            aj = L
        return aj 

    def selectJrand(i,m):
        j=i 
        while (j==i):
            j = int(np.random.uniform(0,m))
        return j
              
    def kernelTrans(X,A,kTup):
        m,n = np.shape(X)
        K = np.zeros((m,1))
        if kTup[0]=='linear': 
            K = X @ A.T   
        elif kTup[0]=='rbf':
            for j in range(m):
                deltaRow = X[j] - A
                K[j] = deltaRow@deltaRow.T
            K = np.exp(K/(-1*kTup[1]**2)).T       
        elif kTup[0]=='laplace':
            for j in range(m):
                K[j]  = np.linalg.norm((X[j] - A),ord=1)
            K = np.exp(K/(-1*kTup[1])).T        
        else: 
            raise NameError('Houston We Have a Problem That Kernel is not recognized')
        return K
        
    def calcEk(self,k):
        fXk = (self.y*self.alphas.T)@self.K[:,k].T + self.b
        Ek = fXk - float(self.y[k])
        return Ek
    
    def updateEk(self, k):
        Ek = self.calcEk(k)
        self.eCache[k] = [1,Ek]  
        
    def selectJ(self,i,Ei):
        maxK = -1; maxDeltaE = 0; Ej = 0
        self.eCache[i] = [1,Ei]  
        validEcacheList = np.nonzero(self.eCache[:,0])[0]
        if (len(validEcacheList)) > 1:
            for k in validEcacheList:   
                if k == i: 
                    continue 
                Ek = self.calcEk(k)
                deltaE = abs(Ei - Ek)
                if (deltaE > maxDeltaE):
                    maxK = k; maxDeltaE = deltaE; Ej = Ek
            return maxK, Ej
        else:   
            j = type(self).selectJrand(i, self.m)
            Ej = self.calcEk(j)
        return j, Ej
        
    def innerL(self,i):
        Ei = self.calcEk(i)
        if ((self.y[i]*Ei < -self.tol) and (self.alphas[i] < self.C)) or ((self.y[i]*Ei > self.tol) and (self.alphas[i] > 0)):
            j,Ej = self.selectJ(i,Ei) 
            alphaIold = self.alphas[i].copy(); alphaJold = self.alphas[j].copy();
            if (self.y[i] != self.y[j]):
                L = max(0, self.alphas[j] - self.alphas[i])
                H = min(self.C, self.C + self.alphas[j] - self.alphas[i])
            else:
                L = max(0, self.alphas[j] + self.alphas[i] - self.C)
                H = min(self.C, self.alphas[j] + self.alphas[i])
            if L==H: 
                return 0
            eta = 2.0 * self.K[i,j] - self.K[i,i] - self.K[j,j]
            if eta >= 0: 
                return 0
            self.alphas[j] -= self.y[j]*(Ei - Ej)/eta
            self.alphas[j] = type(self).clipAlpha(self.alphas[j],H,L)
            self.updateEk(j) 
            b1 = self.b - Ei- self.y[i]*(self.alphas[i]-alphaIold)*self.K[i,i] - self.y[j]*(self.alphas[j]-alphaJold)*self.K[i,j]
            b2 = self.b - Ej- self.y[i]*(self.alphas[i]-alphaIold)*self.K[i,j]- self.y[j]*(self.alphas[j]-alphaJold)*self.K[j,j]
            if (0 < self.alphas[i]) and (self.C > self.alphas[i]): self.b = b1
            elif (0 < self.alphas[j]) and (self.C > self.alphas[j]): self.b = b2
            else: self.b = (b1 + b2)/2.0
            return 1
        else: 
            return 0
                    
    def smoP(self):
        iters=0
        entireSet = True; alphaPairsChanged = 0
        while (iters < self.max_iter) and ((alphaPairsChanged > 0) or (entireSet)):
            alphaPairsChanged = 0
            if entireSet:   
                for i in range(self.m):        
                    alphaPairsChanged += self.innerL(i)
                iters += 1
            else:
                nonBoundIs = np.nonzero((self.alphas> 0) * (self.alphas< self.C))[0]
                for i in nonBoundIs:
                    alphaPairsChanged += self.innerL(i)
                iters += 1
            if entireSet: entireSet = False 
            elif (alphaPairsChanged == 0): entireSet = True  
        return self.b[0],self.alphas
    
    def calcWs(self):
        w=(self.effective_alpha_*self.support_labels_)@self.support_vectors_
        return w

In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X,y=load_breast_cancer().data,load_breast_cancer().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=7)

In [3]:
for i in ['laplace','rbf','linear']:
    my_svm=SVM(C=1,kernel=i,gamma=1.2).fit(X_train,y_train)
    print(i,accuracy_score(y_test,my_svm.predict(X_test)))

laplace 0.9298245614035088
rbf 0.9298245614035088
linear 0.9035087719298246
